In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn import model_selection
from sklearn import metrics
#from xgboost import XGBRegressor
from sklearn.svm import LinearSVR
from sklearn import preprocessing

In [ ]:
df = pd.read_csv("/kaggle/input/tabular-playground-series-aug-2021/train.csv")
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')
df['kfold'] = -1
kfold = model_selection.KFold(n_splits = 5, shuffle = True, random_state = 0)  #5 is more or less a good split value

for fold, ( train_indices, test_indices ) in enumerate(kfold.split(df)):
    df.loc[test_indices,'kfold'] = fold
df_test.drop('id',axis = 1, inplace = True )

cont = [col for col in df.columns if col.startswith('f')]
for col in cont:
    df[f'{col}_bin'], bins = pd.qcut(df[col], q = 4, retbins = True, labels = [0,1,2,3])
    df[f'{col}_bin'] = pd.to_numeric(df[f'{col}_bin'])
    bins = np.concatenate([[-np.Inf], bins[1:-1],[np.Inf]])
    df_test[f'{col}_bin'] = pd.cut(df_test[col],bins, labels = [0,1,2,3])
    df_test[f'{col}_bin'] = pd.to_numeric(df_test[f'{col}_bin'])

scaler = preprocessing.Normalizer()
df[cont] = scaler.fit_transform(df[cont])
df_test[cont] = scaler.transform(df_test[cont])

not_features = ['id','kfold','loss']
final_predictions = []
error_score = []

for fold in range(5):
    
    train_fold = df[df['kfold'] != fold ]
    xtrain = train_fold.drop( not_features, axis = 1 )
    ytrain = train_fold['loss']
    
    valid_fold = df[df['kfold'] == fold ]
    xval = valid_fold.drop( not_features, axis = 1 )
    yval = valid_fold['loss']
    
    #model = XGBRegressor(n_jobs = -1, tree_method = 'gpu_hist', gpu_id = 0, predictor = 'gpu_predictor' )
    #model.fit(xtrain,ytrain)
    clf = LinearSVR(random_state=0, tol=1e-5, max_iter=5000)
    clf.fit(xtrain, ytrain)
    prediction = clf.predict(xval)
    #prediction = model.predict(xval)
    final_predictions.append(model.predict(df_test))
    error = metrics.mean_squared_error(prediction, yval, squared = False )
    print( fold, error )
    error_score.append(error)
    
print(f"Mean : {np.mean(error_score)}\nStandard Deviation : {np.std(error_score)}")

In [ ]:
final_predictions = np.column_stack(final_predictions)
result = final_predictions.mean( axis = 1 )
sample_submission['loss'] = result
sample_submission.to_csv('submission.csv', index = False )